# Rover Project

I had problems with JupyterNotebooks while importing some libraries, so i decided to continue and use PyCharm!
My images and outputs are in the /output folder, together with the video!

So, it goes through all the steps from the notebook given to us as guidance, but since i used an IDE i did not have the luxury to use Ipython previleges. What i did, as i went step by step, and plotet all the methods! The at the end i populated the process_image() method and made a movie with moviepy.

So, here we go...

## IMPORTS

First thing first, importing libraries!
OpenCV for warping and manipulating with images, Numpy is the one library that you esentially use evevrytime you do math with python, Matlotlib for ploting images, Pandas for reading files and creating data(arrays, lists), and lastly the Moveipy to export the data as vide file.

In [1]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
from moviepy.editor import ImageSequenceClip


/usr/local/lib/python3.5/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


## Creating the Databucket Class

Here we create a class that takes data from the images we saved during training mode with RoboSIM. It uses pandas to create a dataframe where we sepcify the location of those images and the csv file.

In [2]:
df = pd.read_csv('../run_two/robot_log.csv')
csv_img_list = df["Path"].tolist()
ground_truth = mpimg.imread('../calibration_images/map_bw.png')
ground_truth_3d = np.dstack((ground_truth*0, ground_truth*255, ground_truth*0)).astype(np.float)

class Databucket():
    def __init__(self):
        self.images = csv_img_list
        self.xpos = df["X_Position"].values
        self.ypos = df["Y_Position"].values
        self.yaw = df["Yaw"].values
        self.count = -5
        self.worldmap = np.zeros((200, 200, 3)).astype(np.float)
        self.ground_truth = ground_truth_3d

## Creating the prespective transform class

Here we use openCV finctions Warp getPerspectiveTransform() and warpPerspective() to map a pixel from camera to a 10cm sauqred in real world and then warping it as it is seen from top of rover (kindof radar/lidar/sonar)

In [ ]:
def perspect_transform(img, src, dst):
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))  # keep same size as input image
    return warped

## Source & Destination picture transform methods

Here is the mapping of above finctions. So in order to do warping we need the surce pixels (as seen  from the camera on the rover) and the destination.
Well, this step is done in the process_image() function, but as i explained, i used an IDE and it was better code managing to make a methods for it and just call it from process_image()

In [ ]:
def sour():
    return np.float32([[14, 140], [301, 140], [200, 96], [118, 96]])
def dest(img, dst_size, bottom_offset):
    return np.float32([[img.shape[1] / 2 - dst_size, img.shape[0] - bottom_offset],
                              [img.shape[1] / 2 + dst_size, img.shape[0] - bottom_offset],
                              [img.shape[1] / 2 + dst_size, img.shape[0] - 2 * dst_size - bottom_offset],
                              [img.shape[1] / 2 - dst_size, img.shape[0] - 2 * dst_size - bottom_offset],])

## Color Thresholding

One of most important function is this, as all the braind of the robot is based on color thresholding! So, we make some functions that will be used to not only determine the terrain and path, but also the rocks and other obstacles. All this based on color of pixels that will see through the camera on the rover!

In [ ]:
def color_terrain(img):
    color_select = np.zeros_like(img[:,:,0])
    above_thresh = (img[:,:,0] > 160) \
                & (img[:,:,1] > 160) \
                & (img[:,:,2] > 160)
    color_select[above_thresh] = 1
    return color_select
def color_obstcls(img):
    color_select = np.zeros_like(img[:,:,0])
    above_thresh = (img[:,:,0] < 160) \
                & (img[:,:,1] < 160) \
                & (img[:,:,2] < 160)
    color_select[above_thresh] = 1
    return color_select
def color_rocks(img):
    lower_thresh = np.array([20, 100, 100])
    upper_thresh = np.array([25, 255, 255])
    color_select = cv2.inRange(cv2.cvtColor(img, cv2.COLOR_RGB2HSV, 3), lower_thresh, upper_thresh)
    return color_select

## Mapping the terrain to world cordinates

In [ ]:
def rover_coords(binary_img):
    ypos, xpos = binary_img.nonzero()
    x_pixel = np.absolute(ypos - binary_img.shape[0]).astype(np.float)
    y_pixel = -(xpos - binary_img.shape[0]).astype(np.float)
    return x_pixel, y_pixel

def rotate_pix(xpix, ypix, yaw):
    yaw_rad = yaw * np.pi / 180
    xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))
    ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
    return xpix_rotated, ypix_rotated

def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale):
    xpix_translated = (xpix_rot / scale) + xpos
    ypix_translated = (ypix_rot / scale) + ypos
    return xpix_translated, ypix_translated

def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
    xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
    xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
    x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
    y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
    return x_pix_world, y_pix_world

## Average Angle and Distance

 Afer we have calculated the x and y positions of navigable terrain pixels in rover space and now we'd like to decide which direction to steer the rover, and to do so, we use Polar Cordinates, as it is a two-dimensional coordinate system in which each point on a plane is determined by a distance from a reference point and an angle from a reference direction.

In [ ]:
def to_polar_coords(x_pixel, y_pixel):
    dist = np.sqrt(x_pixel**2 + y_pixel**2)
    angles = np.arctan2(y_pixel, x_pixel)
    return dist, angles

## Video Export method

Here is a method, that takes images as input (one per time) and makes them together as moveie. Using moviepy :)

In [ ]:
def video_export(process_image):
    output = '../output/test_mapping.mp4'
    clip = ImageSequenceClip(data.images, fps=60)
    new_clip = clip.fl_image(process_image)
    new_clip.write_videofile(output, audio=False)



# The processing method!



The process_image() mathod, is the most iportant one here, as it takes a simple image, and makes all the necesary processing and outputs another image composed of four windowed processed images!

In [ ]:
def process_image(img):
    cam_offset = 6
    world_scale = 10
    world_size = data.worldmap.shape[0]
    #1
    source = sour()
    destination = dest(img, world_scale, cam_offset)
    #2
    warped = perspect_transform(img, source, destination)
    #3
    terrain_select = color_terrain(warped)
    obstcls_select = color_obstcls(warped)
    rocks_select = color_rocks(warped)
    #4
    terrain_xpix, terrain_ypix = rover_coords(terrain_select)
    obstcls_xpix, oobstcls_ypix = rover_coords(obstcls_select)
    rocks_xpix, rocks_ypix = rover_coords(rocks_select)
    #5
    xpos, ypos, yaw = data.xpos[data.count], data.ypos[data.count], data.yaw[data.count]
    terrain_x_world, terrain_y_world = pix_to_world(terrain_xpix, terrain_ypix, xpos, ypos, yaw, world_size, world_scale)
    obstcls_x_world, obstcls_y_world = pix_to_world(obstcls_xpix, oobstcls_ypix, xpos, ypos, yaw, world_size, world_scale)
    rocks_x_world, rocks_y_world = pix_to_world(rocks_xpix, rocks_ypix, xpos, ypos, yaw, world_size, world_scale)
    #6
    data.worldmap[terrain_y_world, terrain_x_world, 0] += 1
    data.worldmap[obstcls_y_world, obstcls_x_world, 2] += 1
    data.worldmap[rocks_y_world, rocks_x_world, 1] += 1

    mymap = data.worldmap.copy()
    mymap[(data.worldmap[:, :, 0] > 1), 0] = 255
    mymap[(data.worldmap[:, :, 2] > 20), 2] = 255
    mymap[(data.worldmap[:, :, 1] > 1), 1] = 255
    
    dist, angles = to_polar_coords(terrain_xpix, terrain_ypix)
    mean_dir = np.mean(angles)

    output_image = np.zeros((img.shape[0] + data.worldmap.shape[0], img.shape[1] * 2, 3))
    # Next you can populate regions of the image with various output
    # Here I'm putting the original image in the upper left hand corner
    output_image[0:img.shape[0], 0:img.shape[1]] = img
    # Let's create more images to add to the mosaic, first a warped image
    warped = perspect_transform(img, source, destination)
    # Add the warped image in the upper right hand corner
    output_image[0:img.shape[0], img.shape[1]:] = warped
    # Overlay worldmap with ground truth map
    map_add = cv2.addWeighted(mymap, 1, data.ground_truth, 0.5, 0)
    # Flip map overlay so y-axis points upward and add to output_image
    output_image[img.shape[0]:, 0:data.worldmap.shape[1]] = np.flipud(map_add)
    output_image[img.shape[0]:img.shape[0] * 2, img.shape[1]:] = np.dstack((obstcls_select * 255,) * 3)

    data.count += 1  # Keep track of the index in the Databucket()
    return output_image

## Calling methods

We instanciate an data class that makes us ready the images with pandas dataset, and then those data are passed to vide_export() that takes those images and passes through process_image() function, and outputs a video.

In [ ]:
data = Databucket()
video_export(process_image)